In [1]:
from typing import Any
from rio_tiler.io import XarrayReader
from titiler.xarray.factory import xarray_open_dataset, update_dataset

from profiler import profile

In [6]:
import morecantile
from rio_tiler.io import XarrayReader


@profile(add_to_return=True, cprofile=True, quiet=True)
def get_tile(src_path: str, x: int, y: int, z: int, *, variable: str, **kwargs: Any):

    with xarray_open_dataset(
        src_path,
        z=z,
        **kwargs,
    ) as dataset:
        
        dataarray, _ = update_dataset(dataset, variable=variable)
        
        with XarrayReader(dataarray) as src_dst:
            return src_dst.tile(x, y, z)
    


s3://veda-data-store-staging/EIS/zarr/FWI-GEOS-5-Hourly (this is a protected data source but the titiler deployment I working with has access, configuring the domain name now...)

s3://power-analysis-ready-datastore/power_901_monthly_meteorology_utc.zarr (publicly available, variable TS)

s3://cmip6-pds/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r2i1p1f1/Amon/tas/gn/v20180827/ (publicly available, variable tas)

In [7]:
_, logs = get_tile(
    "s3://power-analysis-ready-datastore/power_901_monthly_meteorology_utc.zarr", 
    4, 
    4, 
    4, 
    variable="TS",
)

In [8]:
logs

{'GET': 52,
 'Timing': 5.186955213546753,
 'cprofile': ['   ncalls  tottime  percall  cumtime  percall filename:lineno(function)',
  "       40    4.844    0.121    4.844    0.121 {method 'acquire' of '_thread.lock' objects}",
  "        1    0.027    0.027    0.028    0.028 {method 'start' of 'rasterio._env.GDALEnv' objects}",
  '        8    0.017    0.002    0.021    0.003 core.py:2142(_decode_chunk)',
  '        1    0.015    0.015    0.015    0.015 {rasterio._warp._reproject}',
  '      408    0.011    0.000    0.015    0.000 typing.py:1065(_get_protocol_attrs)',
  '113732/112582    0.011    0.000    0.042    0.000 {built-in method builtins.isinstance}',
  '2221/2189    0.010    0.000    0.014    0.000 indexing.py:512(shape)',
  '        9    0.007    0.001    0.007    0.001 crs.py:183(__init__)',
  '     2437    0.006    0.000    0.082    0.000 variable.py:194(as_compatible_data)',
  '39365/33478    0.004    0.000    0.005    0.000 {built-in method builtins.len}']}